In [1]:
import numpy as np
from scipy.stats import binom, poisson

## Problem

Suppose we sample random a random variable $X_i$ as follows:

$$\begin{align} X_i|N_i=n &\sim Bin(n, p) \\
N_i &\sim Poisson(\lambda) \end{align}$$

Generate n samples from this distribution, using post-stratification using the variable $Y$ where $Y=N$ if $N<8$ and $Y=8$ if $N\geq 8$ to estimate $\theta = P(X=0)$.

In [4]:
n = 10_000
lam = 3
p = 0.5

In [5]:
# answer
N = poisson.rvs(lam, size=n)
X = binom.rvs(n=N, p=p)

locs = [N == i for i in range(8)]
locs.append(N>=8)
X_i_bar = np.array([np.mean(X[loc]==0) for loc in locs])
p_i = np.array([poisson.pmf(i, lam) for i in range(8)] + [1 - poisson.cdf(7, lam)])
theta_hat = np.sum(X_i_bar * p_i)

TypeError: Cannot cast array data from dtype('int64') to dtype('int32') according to the rule 'safe'

In [ ]:
import hyperopt
import numpy as np
from scipy.special import expit
from scipy.stats import expon, norm, uniform
import simpy

## do not load any additional libraries

## Problem

Suppose a company has a contract to build 2 charging stations for electric vehicles in a city, each with capacity n_1, n_2, and locations (x_1, y_1), (x_2, y_2). n_1 and n_2 can be integers between 1 and 20, and the x and y coordinates are real numbers between 0 and 1.


(1) The city is laid out on a continous [0,1] x [0,1] grid.

(2) The $i$-th car will need a charge at time $t_i$, which is equal to $t_{i-1} + dt_i$, where $dt_i \sim exp(\lambda)$. At the time they decide to re-charge their car, they will be at location (expit($a_i$), expit($b_i$)), where a_i is a normal distribution with mean 1 and standard deviation 0.5, and b_i is a normal distribution with mean 0 and standard deviation 1. You can use the expit function loaded above.

(3) Once a car realizes it needs to charge, it will proceed to the nearest charging station in terms of manhattan distance (see below). If the car is more than 0.4 units away in manhatthan distance, it will go to another company's charging station. The amount of time it takes to get a location is the manhattan distance divided by 100.

(3) It takes st = 0.01 to charge a car. A car will leave and go to another company's charging station if there are more than 3 cars in line (excluding cars in service).

(4) For each customer that completes a charge, the company makes a revenue of USD 1. The cost of having a station with n charging units and location (x,y) is $f(n, x, y) = 20 * n + 50 * |x - 0.73| + 60 * |y - 0.5| $

Use hyperopt to find the optimal size and location of the two service stations in order to maximize profit. Use common random numbers and use the number of arrivals as a control variate. Each objective function evaluation should use 50 simulations of the system, and use 50 evaluations fmin.

In [ ]:
lambda = 250

def manhattan_distance(x, y):
    return np.sum(np.abs(x - y))


### run a single simulation here

def simulate(x):
    n1 = x[0]
    n2 = x[1]
    x1 = x[2]
    y1 = x[3]
    x2 = x[4]
    y2 = x[5]
    
    
    
    
## objective function
def objective(x):
    n1 = x[0]
    n2 = x[1]
    x1 = x[2]
    y1 = x[3]
    x2 = x[4]
    y2 = x[5]
    
    
    
res = hyperopt.fmin(fn=objective)


In [ ]:
# answer

lam = 250

def manhattan_distance(x, y):
    return np.sum(np.abs(x - y))


### run a single simulation here, return only the revenue

def simulate(x):
    n1 = x[0]
    n2 = x[1]
    x1 = x[2]
    y1 = x[3]
    x2 = x[4]
    y2 = x[5]
    
    xy1 = np.array([x1, y1])
    xy2 = np.array([x2, y2])
    
    def arrivals():
        while True:
            dt = expon.rvs(scale=1/lam)
            yield env.timeout(dt)
            Y.append(1)
            a = expit(norm.rvs(loc=1, scale=0.5))
            b = expit(norm.rvs(loc=0, scale=1))
            loc = np.array([a,b])
            dists = [manhattan_distance(loc, xy1), manhattan_distance(loc, xy1)]
            ind = np.argmin(dists)
            dist = dists[ind]
            if (dist < 0.4):
                env.process(service(ind, dist))
        
        
    def service(ind, dist):
        yield env.timeout(dist / 100)
        if len(stations[ind].queue) <=3:
            rqt = stations[ind].request()
            yield rqt
            yield env.timeout(0.01)
            stations[ind].release(rqt)
            X.append(1)
        
        
    X = []
    Y = []
    env = simpy.Environment()
    stations = [simpy.Resource(env=env, capacity=n1),
                simpy.Resource(env=env, capacity=n2)]
    env.process(arrivals())
    env.run(until=1)
    
    return np.sum(X), np.sum(Y)
    
    
    
## objective function
def objective(x):
    n1 = x[0]
    n2 = x[1]
    x1 = x[2]
    y1 = x[3]
    x2 = x[4]
    y2 = x[5]
    np.random.seed(42)
    revenues = np.zeros(50)
    arrivals = np.zeros(50)
    for i in range(50):
        revenues[i], arrivals[i] = simulate(x)
    c = (-1) * np.cov(revenues, arrivals)[0][1] / np.var(arrivals)
    rev_mod = revenues + c * (arrivals - lam)
    r = np.mean(rev_mod)
    cost = (20 * (x[0] + x[1]) 
            + 50 * np.abs(x1 - 0.73)
            + 50 * np.abs(x2 - 0.73)
            + 60 * np.abs(y1 - 0.5) 
            + 60 * np.abs(y2 - 0.5))
    return cost - r


res = hyperopt.fmin(fn=objective,
                      algo=hyperopt.tpe.suggest,
                      max_evals=50,
                      rstate=np.random.default_rng(42),
                      space=[hyperopt.hp.quniform('n1',1,20,1),
                             hyperopt.hp.quniform('n2',1,20,1),
                             hyperopt.hp.uniform('x1',0,1),
                             hyperopt.hp.uniform('y1',0,1),
                             hyperopt.hp.uniform('x2',0,1),
                             hyperopt.hp.uniform('y2',0,1)])